In [2]:
%load_ext pycozo.ipyext_direct

In [3]:
%cozo_open 'http', '', {'host': 'https://cozo.julep.ai/', "auth": "COZO_API_KEY"}

In [4]:
::relations

/usr/local/lib/python3.10/dist-packages/pycozo/ext_impl.py:61: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  return res.style.applymap(_colour_code_type)


,name,arity,access_level,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers,description
0,beliefs,14,normal,5,9,0,0,0,
1,beliefs:by_belief_id,8,index,8,0,0,0,0,
2,beliefs:by_parent_belief,9,index,9,0,0,0,0,
3,beliefs:embedding_space,18,index,15,3,0,0,0,
4,beliefs:fact_embedding_space,18,index,15,3,0,0,0,
5,beliefs:summary,10,index,6,4,0,0,0,
6,character_principles,4,normal,2,2,0,0,0,
7,characters,7,normal,2,5,0,0,0,
8,entries,9,normal,5,4,0,0,0,
9,entries:by_parent_id,6,index,6,0,0,0,0,


In [8]:
:create characters {
    character_id: Uuid,
    updated_at: Validity default [floor(now()), true],
    =>
    model: String? default null,
    name: String,
    about: String default "",
    metadata: Json default {},
    created_at: Float default now(),
}

,status
0,OK


In [9]:
:create users {
    user_id: Uuid,
    updated_at: Validity default [floor(now()), true],
    =>
    email: String,
    name: String,
    about: String default "",
    metadata: Json default {},
    created_at: Float default now(),
}

,status
0,OK


In [10]:
::index create users:by_email {
    email,
    user_id,
    name,
    about,
    metadata
}

,status
0,OK


In [11]:
:create sessions {
    character_id: Uuid,
    user_id: Uuid? default null,
    session_id: Uuid,
    updated_at: Validity default [floor(now()), true],
    =>
    situation: String,
    summary: String? default null,
    metadata: Json default {},
    created_at: Float default now(),
    ended_at: Float? default null,
}

,status
0,OK


In [18]:
:create entries {
    session_id: Uuid,
    entry_id: Uuid default rand_uuid_v4(),
    timestamp: Float default now(),
    role: String,
    name: String? default null,
    =>
    content: String,
    token_count: Int,
    processed: Bool default false,
    parent_id: Uuid? default null,
}

,status
0,OK


In [19]:
::index create entries:by_parent_id {
    parent_id,
    session_id,
}

,status
0,OK


In [20]:
::index create entries:by_processed {
    processed,
    session_id,
}

,status
0,OK


In [21]:
:create models {
    model_name: String,
    max_length: Int,
    updated_at: Validity default [floor(now()), true],
    =>
    default_settings: Json default {},
}

,status
0,OK


In [4]:
:create beliefs {
    referrent_is_user: Bool,
    referrent_id: Uuid,
    subject_is_user: Bool? default null,
    subject_id: Uuid? default null,
    belief_id: Uuid,
    =>
    belief: String,
    valence: Float default 0.0,
    source_episode: Uuid? default null,
    parent_belief: Uuid? default null,
    sentiment: String default "neutral",
    processed: Bool default false,
    created_at: Float default now(),
    embedding: <F32; 384>,
    fact_embedding: <F32; 1536>? default null,
}


,status
0,OK


In [5]:

::index create beliefs:by_belief_id {
    belief_id,
    belief,
    valence,
    sentiment,
}

,status
0,OK


In [6]:
::index create beliefs:by_parent_belief {
    parent_belief,
    belief_id,
    belief,
    valence,
    sentiment,
}


,status
0,OK


In [7]:
::hnsw create beliefs:embedding_space {
    dim: 384,
    m: 50,
    dtype: F32,
    fields: embedding,
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: true,
    keep_pruned_connections: false,
}


,status
0,OK


In [8]:
::hnsw create beliefs:fact_embedding_space {
    dim: 1536,
    m: 50,
    dtype: F32,
    fields: fact_embedding,
    filter: !is_null(fact_embedding),
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}


,status
0,OK


In [9]:
::fts create beliefs:summary {
    extractor: belief,
    tokenizer: Simple,
    filters: [AsciiFolding, AlphaNumOnly, Lowercase, Stemmer('english'), Stopwords('en')],
}


,status
0,OK


In [10]:
:create episodes {
    referrent_is_user: Bool,
    referrent_id: Uuid,
    subject_is_user: Bool? default null,
    subject_id: Uuid? default null,
    episode_id: Uuid,
    last_accessed_at: Validity default [floor(now()), true],
    =>
    summary: String,
    parent_episode: Uuid? default null,
    happened_at: Float default now(),
    duration: Float default 0,
    importance: Float default 0.5,
    sentiment: String default "neutral",
    sentiment_strength: Float default 0.33334,
    created_at: Float default now(),
    embedding: <F32; 384>,
    fact_embedding: <F32; 1536>? default null,
}



,status
0,OK


In [11]:
::index create episodes:by_episode_id {
    episode_id,
    summary,
    parent_episode,
    happened_at,
    duration,
    importance,
    sentiment,
    sentiment_strength,
    created_at,
}


,status
0,OK


In [12]:
::hnsw create episodes:embedding_space {
    dim: 384,
    m: 50,
    dtype: F32,
    fields: embedding,
    distance: Cosine,
    ef_construction: 20,
    extend_candidates: true,
    keep_pruned_connections: false,
}


,status
0,OK


In [13]:
::hnsw create episodes:fact_embedding_space {
    dim: 1536,
    m: 50,
    dtype: F32,
    fields: fact_embedding,
    distance: Cosine,
    filter: !is_null(fact_embedding),
    ef_construction: 20,
    extend_candidates: false,
    keep_pruned_connections: false,
}



,status
0,OK


In [14]:
::fts create episodes:summary {
    extractor: summary,
    tokenizer: Simple,
    filters: [AsciiFolding, AlphaNumOnly, Lowercase, Stemmer('english'), Stopwords('en')],
}


,status
0,OK


In [5]:
:create user_principles {
    user_id: Uuid,
    full: Bool default false,
    updated_at: Validity default [floor(now()), true],
    =>
    processed_at: Float? default null,
    scores: Json
}

:create character_principles {
    character_id: Uuid,
    updated_at: Validity default [floor(now()), true],
    =>
    processed_at: Float? default null,
    scores: Json
}

:create user_humantic_profile {
    user_id: Uuid,
    updated_at: Validity default [floor(now()), true],
    =>
    processed_at: Float? default null,
    result: Json
}


eval::stored_relation_conflict

  × Stored relation user_humantic_profile conflicts with an existing one
